# The Real code

In [6]:
using Random, Distributions, Plots

What needs to be done:
. Statistics along the way
. Rebalancing ?
. ArrivalProfiles
. Last simulation
... Plots
... Time to run consecutively (does that have something to do with rebalancing?)
... 

Questions for later:
. a total number of runs (a single run has the length of the number of days per run) ??
. Struct for e?
. The runtime, and rebalancing time
... Pretty much just rebalancing!
. what to do when travel time passes total time?

In [42]:
# Using tuples instead of PriorityQueue, because they can have multiple identical keys and the queue cannot.
# This is slightly slower but not by that much, not really a noticable difference in this case. 
# You have to get real up there in iterations before it really chugs behind.

# mutable struct Events
#     e::Array{Tuple{Float64,SubString},1}
# end
# Figure out if this can be a struct !!!

struct Environment
    hours::Float64
    lambda::Float64  #arrival rate
    mu::Float64 #service rate
   
   #Add arrival rate per hour for both stations. Add travel-time rate for both stations.
    Environment() = new(
    2, 
    (1/6), 
    (1/12)
    )
end

mutable struct Stats
    n_cars_st1::Int64
    n_cars_st2::Int64
    unfilled_st1::Int64
    unfilled_st2::Int64
  
end

# Might not be just as fast as PriorityQueue but it is not that much behind
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end

insert_and_sort! (generic function with 1 method)

In [43]:
function arrivals_generator(e::Array{Tuple{Float64,String},1},env::Environment)

    # Arrivals at station 1
    push!(e,(rand(Exponential(env.lambda)),"st1")) # First arrival
    while last(e)[1] <= env.hours
       push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st1"))
    end

    # Arrivals at station 2 (repeat of station 1)
    push!(e,(rand(Exponential(env.lambda)),"st2"))
    while last(e)[1] <= env.hours
        push!(e,(rand(Exponential(env.lambda))+last(e)[1],"st2"))
    end


    # Sorting the arrivals so first time is 1 and so on
    sort!(e)

    return e
end

arrivals_generator (generic function with 3 methods)

In [59]:
# Do this:
# stats.overview_usage = [stats.overview_usage; t state.queue_length state.server_busy]
# Real nice, steal that shit!

function arrival(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)
    # WHAT to do when travel time passes total time?? makes sense? ask teacher
    if station == "st1"
        # Student arrives at empty station. That is one unsatisfied costumer :(
        if stat.n_cars_st1 == 0
            stat.unfilled_st1 += 1
            popfirst!(e)
        else
            stat.n_cars_st1 -= 1
        
            insert_and_sort!(e,(rand(Exponential(env.mu))+e[1][1],"que_st2") )
            popfirst!(e)
        end
        
    elseif station == "st2"
        if stat.n_cars_st2 == 0
            stat.unfilled_st2 += 1
            popfirst!(e)
        else
            stat.n_cars_st2 -= 1
            
            insert_and_sort!(e,(rand(Exponential(env.mu))+e[1][1],"que_st1") )
            popfirst!(e)
        end

    else
        throw(ErrorException("Something went wrong in sim()."))
    end

end 

function travel(station::String,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if station == "que_st1"
        stat.n_cars_st1 += 1
        popfirst!(e)
    elseif station == "que_st2"
        stat.n_cars_st2 += 1
        popfirst!(e)
    else 
        throw(ErrorException("Something went wrong in sim()."))
    end

end


travel (generic function with 3 methods)

In [66]:
function sim(env::Environment,stat::Stats)
    e = Array{Tuple{Float64,String},1}()
    arrivals_generator(e,env)
    
    # Missing statistics all the way through
    while !isempty(e)
        load_type = e[1][2]

        if load_type == "st1" || load_type == "st2"
            arrival(load_type,e,env,stat)
        elseif load_type == "que_st1" || load_type == "que_st2"
            travel(load_type,e,env,stat)
        else
            throw(ErrorException("Issue in the event type!"))
        end
    end
    return stat.n_cars_st1,stat.n_cars_st2,stat.unfilled_st1,stat.unfilled_st2
end

sim (generic function with 2 methods)

### Testing

In [132]:
env = Environment()
stat = Stats(5,5,0,0)
sim(env,stat)

(2, 8, 3, 0)

In [47]:
env = Environment()
e = Array{Tuple{Float64,String},1}()
arrivals_generator(e,env)
e[1][2]

"st1"

In [4]:
v = Array{Tuple{Float64,String},1}([(1,"1"),(4,"2"),(223,"3")])
sort!(v)
insert_and_sort!(v, (2,"4"))
v

5-element Vector{Tuple{Float64, String}}:
 (1.0, "1")
 (2.0, "4")
 (4.0, "2")
 (123.0, "5")
 (223.0, "3")